# Assignment 1: Time Series Forecast With Python (Seasonal ARIMA)

**Lecturer**: Vincent Claes<br>
**Authors:** Bryan Honof, Jeffrey Gorissen<br>
**Start Date:** 19/10/2018
    
**Objective:** Visualize and predict the future temperatures via ARIMA

**Description:** In this notebook we train our model

In [1]:
import math
import warnings
import datetime

import pandas            as pd
import itertools         as it
import statsmodels.api   as sm
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore") # specify to ignore warning messages

In [2]:
data_csv = pd.read_csv('./data/rolmean_data.csv')
data = pd.DataFrame()

# Convert the creation_date column to datetime64
data['dateTime'] = pd.to_datetime(data_csv['dateTime'])
# Convert the value column to float
data['temperature'] = pd.to_numeric(data_csv['temperature'])

# Set the dateTime column as index
data = data.set_index(['dateTime'])

# Sort the dataFrame just to be sure...
data = data.sort_index()

data = data.dropna()

# Double check the results
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 828 entries, 2018-11-11 03:00:00 to 2018-11-19 17:45:00
Data columns (total 1 columns):
temperature    828 non-null float64
dtypes: float64(1)
memory usage: 12.9 KB


In [3]:
data.tail(5)

,temperature
dateTime,
2018-11-19 16:45:00,17.28
2018-11-19 17:00:00,17.29
2018-11-19 17:15:00,17.23
2018-11-19 17:30:00,17.21
2018-11-19 17:45:00,17.03


In [4]:
# Tune Seasonal ARIMA model
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0,2)

# Generate all different combinations of p, q and q triplets
pdq = list(it.product(p, d, q))
print(pdq)

# Generate all different combinations of seasonal p, q and q triplets
# Seasonality is one week (24*7 = 168 hours)
seasonal_pdq = [(x[0], x[1], x[2], 15) for x in list(it.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (1, 1, 1)]
Examples of parameter combinations for Seasonal ARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 15)
SARIMAX: (0, 0, 1) x (0, 1, 0, 15)
SARIMAX: (0, 1, 0) x (0, 1, 1, 15)
SARIMAX: (0, 1, 0) x (1, 0, 0, 15)


In [5]:
result_list = []
AIC         = []
parm_       = []
parm_s      = []

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(data,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            AIC.append(results.aic)
            parm_.append(param)
            parm_s.append(param_seasonal)
            
            print('ARIMA{}x{} - AIC:{}'.format(param, param_seasonal, round(results.aic,2)))
            result_list.extend([param, param_seasonal, round(results.aic,2)])
        except:
            print('error')
            continue
            
print('Done!')

ARIMA(0, 0, 0)x(0, 0, 0, 15) - AIC:7155.25
ARIMA(0, 0, 0)x(0, 0, 1, 15) - AIC:5989.39
ARIMA(0, 0, 0)x(0, 1, 0, 15) - AIC:2513.3
ARIMA(0, 0, 0)x(0, 1, 1, 15) - AIC:2472.83
ARIMA(0, 0, 0)x(1, 0, 0, 15) - AIC:2514.3
ARIMA(0, 0, 0)x(1, 0, 1, 15) - AIC:2504.94
ARIMA(0, 0, 0)x(1, 1, 0, 15) - AIC:2479.73
ARIMA(0, 0, 0)x(1, 1, 1, 15) - AIC:2465.37
ARIMA(0, 0, 1)x(0, 0, 0, 15) - AIC:6016.44
ARIMA(0, 0, 1)x(0, 0, 1, 15) - AIC:4944.14
ARIMA(0, 0, 1)x(0, 1, 0, 15) - AIC:1529.65
ARIMA(0, 0, 1)x(0, 1, 1, 15) - AIC:1514.27
ARIMA(0, 0, 1)x(1, 0, 0, 15) - AIC:1531.97
ARIMA(0, 0, 1)x(1, 0, 1, 15) - AIC:1531.69
ARIMA(0, 0, 1)x(1, 1, 0, 15) - AIC:1517.12
ARIMA(0, 0, 1)x(1, 1, 1, 15) - AIC:1512.84
ARIMA(0, 1, 0)x(0, 0, 0, 15) - AIC:-1238.58
ARIMA(0, 1, 0)x(0, 0, 1, 15) - AIC:-1348.54
ARIMA(0, 1, 0)x(0, 1, 0, 15) - AIC:-475.85
ARIMA(0, 1, 0)x(0, 1, 1, 15) - AIC:-1126.81
ARIMA(0, 1, 0)x(1, 0, 0, 15) - AIC:-1271.25
ARIMA(0, 1, 0)x(1, 0, 1, 15) - AIC:-1402.21
ARIMA(0, 1, 0)x(1, 1, 0, 15) - AIC:-756.1
ARIMA(0, 

In [7]:
min(AIC)
pos = AIC.index(min(AIC))
print(parm_[pos], parm_s[pos], min(AIC))

(1, 1, 1) (0, 0, 1, 15) -2121.426066178455
